In [2]:
# Library
import pandas as pd
import tkinter as tk
from tkinter import scrolledtext
from rapidfuzz import process  # Import fuzzy matching library

# Load dataset from Excel file
file_path = "QUESTION RESPONSE CHATBOT.xlsx"
xls = pd.ExcelFile(file_path)

# Combine data from all sheets into a single dataset
dataframes = []
for sheet in xls.sheet_names:
    df = pd.read_excel(xls, sheet_name=sheet, usecols=[1, 2], names=["Question", "Response"])
    dataframes.append(df)
dataset = pd.concat(dataframes, ignore_index=True)

# Extract questions for fuzzy matching
questions_list = dataset["Question"].astype(str).tolist()

# Function to get chatbot response with fuzzy matching
def get_response(user_input):
    user_input = user_input.strip().lower()
    # Find the most similar question
    match, score, index = process.extractOne(user_input, questions_list)
    if score > 75:  # Accept if similarity is above threshold
        response = dataset.iloc[index]["Response"]
        return response
    return "I'm sorry, I don't understand that question."

# Function to handle user input and display conversation
def send_message(event=None):  # Accept event parameter for key binding
    user_text = entry.get()
    if user_text:
        chat_area.insert(tk.END, f"You: {user_text}\n", "user")
        response = get_response(user_text)
        chat_area.insert(tk.END, f"Bot: {response}\n", "bot")
        entry.delete(0, tk.END)
        chat_area.yview(tk.END)

# Function to properly close the application
def on_closing():
    root.destroy()
    root.quit()

# UI Setup
root = tk.Tk()
root.title("ChatCotz Bot")
root.geometry("500x600")
root.configure(bg="#e3f2fd")  # Light blue background
root.protocol("WM_DELETE_WINDOW", on_closing)  # Ensure the app closes completely

# Styling Variables
FONT = ("Arial", 12)
USER_COLOR = "#1e88e5"  # Blue for user messages
BOT_COLOR = "#43a047"  # Green for bot messages
BUTTON_COLOR = "#0288d1"
TEXT_COLOR = "#212121"

# Create a frame for chat area
chat_frame = tk.Frame(root, bg="#e3f2fd")
chat_frame.pack(pady=10, fill=tk.BOTH, expand=True)

# Create chat area with scrollable text
chat_area = scrolledtext.ScrolledText(chat_frame, wrap=tk.WORD, state=tk.NORMAL, font=FONT, bg="#ffffff", fg=TEXT_COLOR, width=50, height=20)
chat_area.tag_configure("user", foreground=USER_COLOR, font=("Arial", 12, "bold"))
chat_area.tag_configure("bot", foreground=BOT_COLOR, font=("Arial", 12))
chat_area.pack(padx=10, pady=5, fill=tk.BOTH, expand=True)

# Create input field frame
entry_frame = tk.Frame(root, bg="#e3f2fd")
entry_frame.pack(pady=5, fill=tk.X)
entry = tk.Entry(entry_frame, font=FONT, bg="#ffffff", fg=TEXT_COLOR, bd=2, relief=tk.GROOVE)
entry.pack(side=tk.LEFT, padx=10, pady=5, fill=tk.BOTH, expand=True)
entry.bind("<Return>", send_message)  # Bind Enter key to send message

# Create send button
send_button = tk.Button(entry_frame, text="Send", command=send_message, font=("Arial", 12, "bold"), bg=BUTTON_COLOR, fg="white", padx=15, pady=5, relief=tk.FLAT)
send_button.pack(side=tk.RIGHT, padx=10)

# Run the chatbot application
root.mainloop()